In [10]:
import numpy as np
import torch

from torch import nn
from torch.nn import functional as F
from PIL import Image
from torchvision import transforms, datasets
from tqdm import tqdm
from torchvision import models

from sklearn.model_selection import train_test_split

Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) сверточной сетью (самописной)

In [11]:
dataset = datasets.CIFAR100(root='data/', train=True, download=True)

def train_valid_split(Xt):
    X_train, X_test = train_test_split(Xt, test_size=0.25, random_state=13)
    return X_train, X_test

class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]
    
trans_actions = transforms.Compose([transforms.Scale(44),
                                    transforms.RandomCrop(32, padding=4), 
                                    transforms.ToTensor()])

train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, trans_actions)
valid_dataset = MyOwnCifar(valid_dataset, transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=3)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=1)

Files already downloaded and verified


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [12]:
class MyCNN(nn.Module):

  def __init__(self):
    super(MyCNN, self).__init__()
    self.dp_one = nn.Dropout(0.2)
       
    self.bn_one = torch.nn.BatchNorm2d(3) 
    self.conv_one = torch.nn.Conv2d(3, 8, 3)

    self.bn_two = torch.nn.BatchNorm2d(8) 
    self.conv_two = torch.nn.Conv2d(8, 64, 3)

    self.bn_three = torch.nn.BatchNorm2d(64)

    self.fc1 = torch.nn.Linear(2304, 500)
    self.out = torch.nn.Linear(500, 100)
        
        
  def forward(self, x):
    x = self.bn_one(x)
    x = self.conv_one(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
        
    x = self.bn_two(x)
    x = self.conv_two(x)
    x = F.leaky_relu(x)
    x = F.max_pool2d(x, 2)
        
    x = self.bn_three(x)
    x = x.view(x.size(0), -1)

    x = self.dp_one(x)
    x = self.fc1(x)
    x = F.relu(x)

    return self.out(x)
       
mycnn = MyCNN()

In [13]:
optimizer = torch.optim.Adam(mycnn.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [14]:
for epoch in tqdm(range(10)):  
    mycnn.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]
        optimizer.zero_grad()

        outputs = mycnn(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    mycnn.eval()
    loss_accumed = 0
    for X, y in valid_loader:
        output = mycnn(X)
        loss = criterion(output, y)
        loss_accumed += loss
    print("Epoch {} valid_loss {}".format(epoch, loss_accumed))

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 10%|█         | 1/10 [00:37<05:37, 37.52s/it]

Epoch 0 valid_loss 402.72607421875


 20%|██        | 2/10 [01:18<05:17, 39.69s/it]

Epoch 1 valid_loss 380.6664123535156


 30%|███       | 3/10 [02:00<04:44, 40.70s/it]

...

Epoch 7 valid_loss 359.023681640625


 90%|█████████ | 9/10 [06:11<00:41, 41.89s/it]

Epoch 8 valid_loss 384.69793701171875


100%|██████████| 10/10 [06:58<00:00, 41.85s/it]

Epoch 9 valid_loss 365.0069580078125
Training is finished!


Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50

In [15]:
resnet50 = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [16]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [17]:
set_parameter_requires_grad(resnet50, True)
resnet50.fc = nn.Linear(2048, 100)

In [18]:
trans_actions = transforms.Compose([transforms.Scale(256),
                                    transforms.RandomCrop(224, padding=4),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])

train_dataset, valid_dataset = train_valid_split(dataset)

train_dataset = MyOwnCifar(train_dataset, trans_actions)
valid_dataset = MyOwnCifar(valid_dataset, valid_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=3)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=128,
                          shuffle=False,
                          num_workers=1)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [19]:
params_to_update = []
for name,param in resnet50.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

optimizer = torch.optim.Adam(params_to_update, lr=0.001)
criterion = nn.CrossEntropyLoss()

In [11]:
for epoch in tqdm(range(2)):  
    resnet50.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]
        optimizer.zero_grad()

        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    resnet50.eval()
    loss_accumed = 0
    for X, y in valid_loader:
        output = resnet50(X)
        loss = criterion(output, y)
        loss_accumed += loss
    print("Epoch {} valid_loss {}".format(epoch, loss_accumed))

print('Training is finished!')

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 50%|█████     | 1/2 [2:09:30<2:09:30, 7770.59s/it]

Epoch 0 valid_loss 1032.6712646484375


100%|██████████| 2/2 [4:20:08<00:00, 7804.10s/it]

Epoch 1 valid_loss 1324.541015625
Training is finished!


Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50 с аугментацией (самописной, с использованием Pytorch встроенных методов)

In [20]:
resnet50_2 = models.resnet50(pretrained=True)

In [21]:
class AddResnet(nn.Module):

  def __init__(self):
    super(AddResnet, self).__init__()

    self.fc1 = torch.nn.Linear(2048, 500)
    self.out = torch.nn.Linear(500, 100)
        
        
  def forward(self, x):

    x = self.fc1(x)
    x = F.relu(x)

    return self.out(x)
       


In [22]:
set_parameter_requires_grad(resnet50_2, True)
resnet50_2.fc = AddResnet()

In [23]:
params_to_update = []
for name,param in resnet50_2.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

optimizer = torch.optim.Adam(params_to_update, lr=0.001)
criterion = nn.CrossEntropyLoss()

In [24]:
for epoch in tqdm(range(2)):  
    resnet50_2.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]
        optimizer.zero_grad()

        outputs = resnet50_2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    resnet50_2.eval()
    loss_accumed = 0
    for X, y in valid_loader:
        output = resnet50_2(X)
        loss = criterion(output, y)
        loss_accumed += loss
    print("Epoch {} valid_loss {}".format(epoch, loss_accumed))

print('Training is finished!')

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 50%|█████     | 1/2 [2:24:59<2:24:59, 8699.65s/it]

Epoch 0 valid_loss 1075.2767333984375


100%|██████████| 2/2 [4:46:56<00:00, 8608.13s/it]

Epoch 1 valid_loss 1264.3275146484375
Training is finished!
